In [ ]:
import pandas as pd

In [ ]:
trueData=pd.read_csv('True.csv')
fakeData=pd.read_csv('Fake.csv')
print(trueData.head(),fakeData.head())

In [ ]:
trueData['label'] = 1
fakeData['label'] = 0
data = pd.concat([trueData, fakeData])
data

In [ ]:
#Data visualization using Seaborn
import seaborn as sbn
sbn.set_style("darkgrid")
sbn.countplot(data['label'])

In [ ]:
data.isnull().sum()

In [ ]:

import matplotlib.pyplot as plt
data['subject'].value_counts()
plt.figure(figsize = (10,10))
sbn.set_style("darkgrid")
sbn.countplot(data['subject']);

In [ ]:
plt.figure(figsize = (10,10))
sbn.set_style("dark")
chart = sbn.countplot(x = "label", hue = "subject" , data = data)
chart.set_xticklabels(chart.get_xticklabels(),rotation=90)

In [ ]:

data['text'] = data['title'] + " " + data['text']
data = data.drop(['title', 'subject', 'date'], axis=1)

In [ ]:
from nltk.corpus import stopwords
from wordcloud import WordCloud

wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords.words('english'), 
                min_font_size = 10).generate(" ".join(data[data['label'] == 0].text)) 
  
# plot the word cloud for fake news data                      
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
print("wordcloud for fake or spam data")
plt.show() 

In [ ]:
from wordcloud import WordCloud

wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords.words('english'), 
                min_font_size = 10).generate(" ".join(data[data['label'] == 1].text)) 
  
# plot the WordCloud image for genuine news data                     
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
print("True or fact data wordcloud")
plt.show()

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(data['text'],data['label'],test_size=0.2, random_state = 1)


In [ ]:
#Multinomial NB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import sklearn.metrics as metrics                                                 
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

pipe = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('clf', MultinomialNB())
])

mnbModel = pipe.fit(x_train, y_train)
prediction = mnbModel.predict(x_test)

mnbScore = metrics.accuracy_score(y_test, prediction)
print("accuracy using multinomial naive bayes:   %0.3f" % (mnbScore*100))
cm = metrics.confusion_matrix(y_test, prediction, labels=[0,1])
fig, ax = plot_confusion_matrix(conf_mat=confusion_matrix(y_test, prediction),
                                show_normed=True,
                                colorbar=True)
plt.show()

In [ ]:
#SVM
from sklearn.svm import LinearSVC
pipe = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('clf', LinearSVC())
])

svmModel = pipe.fit(x_train, y_train)
prediction = svmModel.predict(x_test)

svmScore = metrics.accuracy_score(y_test, prediction)
print("Accuracy using Support vector machine:   %0.3f" % (svmScore*100))
cm = metrics.confusion_matrix(y_test, prediction, labels=[0,1])



fig, ax = plot_confusion_matrix(conf_mat=confusion_matrix(y_test, prediction),
                                show_normed=True, colorbar=True
                                )
plt.show()

In [ ]:
#Passive Aggressive Classifier
from sklearn.linear_model import PassiveAggressiveClassifier
pipe = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf',  PassiveAggressiveClassifier())
])

pacModel = pipe.fit(x_train, y_train)
prediction = pacModel.predict(x_test)

pacScore = metrics.accuracy_score(y_test, prediction)
print("Accuracy using Passive aggressive classifier:   %0.3f" % (pacScore*100))
cm = metrics.confusion_matrix(y_test, prediction, labels=[0,1])



fig, ax = plot_confusion_matrix(conf_mat=confusion_matrix(y_test, prediction),
                                show_normed=True,
                                colorbar=True)
plt.show()

:::::::::GLOVE:::::::::::::

In [ ]:
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Bidirectional
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [ ]:
vocab_size = 100000
oov_token = "<OOV>"
max_length = 100
padding_type = "post"
trunction_type="post"
print(data)
news_text=data.text
labels = data.label
(train_texts,valid_texts,train_labels,valid_labels)=train_test_split(news_text, labels, test_size=0.3)


In [ ]:

tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(train_texts)

In [ ]:

X_train_sequences = tokenizer.texts_to_sequences(train_texts)
print(X_train_sequences)
#X_train_sequences[10:15]

In [ ]:

X_train_padded = pad_sequences(X_train_sequences,maxlen=max_length, padding=padding_type, 
                       truncating=trunction_type)

In [ ]:
X_test_sequences = tokenizer.texts_to_sequences(valid_texts)
X_test_padded = pad_sequences(X_test_sequences,maxlen=max_length, 
                               padding=padding_type, truncating=trunction_type)

In [ ]:
embeddings_index = {}
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print("Found %s word vectors"%len(embeddings_index))

In [ ]:
word_index = tokenizer.word_index
embedding_matrix = np.zeros((len(word_index) + 1, max_length))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
print(embedding_matrix)

In [ ]:

embedding_layer = Embedding(len(word_index) + 1,
                            max_length,
                            weights=[embedding_matrix],
                            input_length=max_length,
                            trainable=False)


In [ ]:
embedding_dim = 16
input_length = 100
model = Sequential([
  embedding_layer,
  Bidirectional(LSTM(embedding_dim, return_sequences=True)),
  Bidirectional(LSTM(embedding_dim,)),
  Dense(6),
  Dense(1)
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
num_epochs=3
history = model.fit(X_train_padded, train_labels, epochs=num_epochs, validation_data=(X_test_padded, valid_labels))
loss, accuracy = model.evaluate(X_test_padded, valid_labels)
print('Accuracy: %f' % (accuracy*100))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

Graphical user interface using Tkinter

In [ ]:
from tkinter import *
import tkinter
from tabulate import tabulate
from collections import defaultdict
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
from PIL import ImageTk,Image
class MyWindow:
    def __init__(self, win):
        self.lbl1=Label(win, text='Enter the news ')
        self.t1=Entry(bd=4)
        self.lbl1.place(x=100, y=30)
        self.t1.place(x=270, y=30)
        self.b1=Button(win,bg='darkcyan',fg='white',activebackground='green' ,text='Check on SVM',command=self.svmModel)
        self.b2=Button(win,bg='darkcyan',fg='white',activebackground='green' ,text='Check on MNB',command=self.mnbModel)
        self.b3=Button(win,bg='darkcyan',fg='white',activebackground='green' ,text='Check on PAC',command=self.pacModel)
        self.b4=Button(win,bg='darkcyan',fg='white',activebackground='green' ,text='Check on GLOVE',command=self.gloveModel)
        self.b5=Button(win,bg='darkcyan',fg='white',activebackground='green', text='Positive Word Frequency', command=self.positiveWordFrequency)
        self.b6=Button(win,bg='darkcyan',fg='white',activebackground='green', text='NegativeWord Frequency', command=self.negativeWordFrequency)
        self.b1.place(x=100, y=90)
        self.b2.place(x=230, y=90)
        self.b3.place(x=360, y=90)
        self.b4.place(x=490, y=90)
        self.b5.place(x=640, y=90)
        self.b6.place(x=840, y=90)


    def svmModel(self):
       value=self.t1.get()
       result=svmModel.predict([value])
       image1 = Image.open("true.jpeg")
       if(result[0]==0):
           image1 = Image.open("fake.jpeg")

       image1= image1.resize((800,605), Image.ANTIALIAS)
       test = ImageTk.PhotoImage(image1)
       self.label1 = tkinter.Label(image=test)
       self.label1.image = test
       self.label1.place(x=170, y=200)
      

    def mnbModel(self):
       value=self.t1.get()
       result=mnbModel.predict([value])
       image1 = Image.open("true.jpeg")
       if(result[0]==0):
           image1 = Image.open("fake.jpeg")

       image1= image1.resize((800,605), Image.ANTIALIAS)
       test = ImageTk.PhotoImage(image1)
       label1 = tkinter.Label(image=test)
       label1.image = test
       label1.place(x=170, y=200)
      

    def pacModel(self):
       value=self.t1.get()
       result=pacModel.predict([value])
       image1 = Image.open("true.jpeg")
       if(result[0]==0):
           image1 = Image.open("fake.jpeg")

       image1= image1.resize((800,605), Image.ANTIALIAS)
       test = ImageTk.PhotoImage(image1)
       label1 = tkinter.Label(image=test)
       label1.image = test
       label1.place(x=170, y=200)
     

    def gloveModel(self):
       value=self.t1.get()
       valueSequence = tokenizer.texts_to_sequences([value])
       padValue= pad_sequences(valueSequence,maxlen=max_length, 
                               padding=padding_type, truncating=trunction_type)
       result=model.predict(padValue)
       image1 = Image.open("true.jpeg")
       if(result[0]<0.5):
           image1 = Image.open("fake.jpeg")

       image1= image1.resize((800,605), Image.ANTIALIAS)
       test = ImageTk.PhotoImage(image1)
       label1 = tkinter.Label(image=test)
       label1.image = test
       label1.place(x=170, y=200)
  
       

    def positiveWordFrequency(self):
        try:
            self.canvas.get_tk_widget().pack_forget()
        except AttributeError:
            pass
       # self.textLabel1.destroy()
        fig = Figure(figsize=(12,10))
        a = fig.add_subplot(111)
        
        wordcloud = WordCloud(width = 1800, height = 1800, 
                 background_color ='white', 
                 stopwords = stopwords.words('english'), 
                 min_font_size = 10).generate(" ".join(data[data['label'] == 1].text)) 
        a.imshow(wordcloud,interpolation='bilinear')
        a.axis('off')
        self.canvas = FigureCanvasTkAgg(fig, master=window)
        self.canvas.get_tk_widget().pack( expand=1)
        self.canvas.draw()

    def negativeWordFrequency(self):
        try:
            self.canvas.get_tk_widget().pack_forget()
        except AttributeError:
            pass
       # self.textLabel1.destroy()
        fig = Figure(figsize=(10,8))
        a = fig.add_subplot(111)
        
        wordcloud = WordCloud(width = 1800, height = 1800, 
                 background_color ='white', 
                 stopwords = stopwords.words('english'), 
                 min_font_size = 10).generate(" ".join(data[data['label'] == 0].text)) 
        a.imshow(wordcloud,interpolation='bilinear')
        a.axis('off')
        self.canvas = FigureCanvasTkAgg(fig, master=window)
        self.canvas.get_tk_widget().pack( expand=1)
        self.canvas.draw()

       





window=Tk()
mywin=MyWindow(window) 
window.title('Fake news detection system')
window.configure(bg='skyblue')
window.geometry("1200x1000+10+10")

window.mainloop()